# NLP Project Pt. 4: Topic Modeling

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth',250)
corpus_df = pd.read_pickle('corpus_df_spacy3.pkl')
corpus_df.reset_index(inplace=True, drop=True)
corpus_df.head()

,URL,TITLE,AUTHOR,DATE,TEXT,WORDCOUNT,TEXT_NOUNS,TEXT_NOUNS_ADJS,TEXT_NOUNS_VERBS,TEXT_NOUNS_VERBS_ADJS,TEXT_ENTS
0,https://www.newyorker.com/magazine/2022/02/14/annunciation,ANNUNCIATION,LAUREN GROFF,"February 7, 2022","['Some nights, in my dreams, I find myself running through those hills above Palo Alto again. It is always just before dawn, and as I run I smell the sun-crisped fields, the sage, the eucalyptus. The mist falls in starched sheets over the distant...",9373,night dream hill dawn sun field sage eucalyptus mist sheet hill footstep breath peloton cyclist morning fog neighborhood river asphalt road oak grace block time eye pool house moss bougainvillea fern year life parent college graduation dozen carn...,night dream hill dawn sun field sage eucalyptus mist starched sheet distant hill footstep breath peloton cyclist morning fog quiet wealthy neighborhood black river asphalt road great strong armed oak grace block time eye converted pool house moss...,night dream find run hill dawn run smell sun crisp field sage eucalyptus mist fall sheet hill press hear footstep breath peloton cyclist whir morning fog swallow descend neighborhood river asphalt road flatten fly oak spread grace block time awak...,night dream find run hill dawn run smell sun crisp field sage eucalyptus mist fall starched sheet distant hill press hear footstep breath peloton cyclist whir morning fog swallow descend quiet wealthy neighborhood black river asphalt road flatten...,"[Palo Alto, Bay, Mountain View, New England, San Francisco, Chinatown, San Francisco, Redwood City, Mountain View, Titania, Germany, Feuerzangenbowle, Redwood City, Mountain View, New York, Caribbean, the British Virgin Islands, Redwood City, Red..."
1,https://www.newyorker.com/magazine/2022/02/07/once-removed,ONCE REMOVED,ALEXANDER MACLEOD,"January 31, 2022","['She did not want to visit the old lady.', 'Amy studied the stroller, then the bags, then her boyfriend and the baby. She checked her phone: 11:26a.m. It was time to go. Ninety degrees, ninety-per-cent humidity, and, according to Google, more th...",7778,lady stroller bag boyfriend baby phone time degree cent humidity hour way stage icon event minute end min min bus min walk min effort afternoon abort mission baby phone number furnace shoulder humming trick nap eye breathing rivulet drool spine p...,old lady stroller bag boyfriend baby phone time degree cent humidity hour way stage icon olympic event separate minute end min min bus min walk min worth effort hot afternoon abort mission sorry baby right phone number furnace shoulder humming tr...,want visit lady study stroller bag boyfriend baby check phone time degree cent humidity accord hour way stage icon event minute break total end walk min train min bus min walk min effort afternoon abort mission abort baby phone number furnace h...,want visit old lady study stroller bag boyfriend baby check phone time degree cent humidity accord hour way stage icon olympic event separate minute break total end walk min train min bus min walk min worth effort hot afternoon abort mission abor...,"[Inverness County, Nova Scotia, Ontario, Toronto, Montreal, Cape Breton, Turkey, Niagara Falls, Montreal]"
2,https://www.newyorker.com/magazine/2022/01/31/long-distance,LONG DISTANCE,AYSEGUL SAVAS,"January 24, 2022","['Lea changed the sheets when she got up. She’d bought flowers the previous day, tulips that she’d put on the dresser. There were carnations on the kitchen table, in a squat glass vase. She thought they looked cheerful, and not too fussy.', 'The ...",4866,sheet flower day tulip dresser carnation kitchen table glass vase fridge thing olive jam prosciutto cheese wine beer cookie bread cracker café place mind scene bed people mess plate idea indulgence sheet light hour imagination afternoon reason be...,sheet flower previous day tulip dresser carnation kitchen table squat glass vase cheerful fussy fridge thing able olive jam prosciutto cheese wine beer cookie bread round taralli cr

# Vectorization

In [292]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF

#function from course notebooks

def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print([[feature_names[i], topic[i].round(3)] for i in topic.argsort()[:-no_top_words - 1:-1]])
    print("\n")
    return model, feature_names, no_top_words

## NMF

In [152]:
#NMF with CV on original text
docs = corpus_df.TEXT
cv = CountVectorizer(stop_words='english', min_df=0.05, max_df=0.6, ngram_range=(1,2))
doc_term = cv.fit_transform(docs)
nmf = NMF(7)
nmf.fit(doc_term)

output = display_topics(nmf, cv.get_feature_names(), 20)

/Users/ami/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  1
parents, phone, town, son, city, road, story, husband, watched, apartment, daughter, boys, job, doesn, baby, girls, sleep, wall, dinner, book

Topic  2
says, doesn, asks, looks, thinks, takes, goes, comes, knows, turns, feels, makes, mark, gets, isn, tells, husband, wants, sees, guy

Topic  3
jack, olive, dog, phone, road, kids, corn, farmer, mom, girls, son, ain, yeah, sky, shit, hello, question, quickly, needed, sun

Topic  4
mr, mrs, feather, fish, hat, dad, bus, boys, line, box, grace, arm, cap, office, lady, corner, shop, ran, road, driver

Topic  5
paul, crystal, mom, dad, god, baby, kids, fine, son, guy, actually, nice, babies, kid, loved, office, happy, ma, sorry, wait

Topic  6
sister, cat, older, baby, uncle, mouse, girls, dog, brother, husband, eat, god, mama, couch, st, food, lady, sugar, watched, yellow

Topic  7
james, mrs, son, restaurant, willing, asks, mary, store, doctor, baby, doesn, young man, daughter, thinks, sleep, land, teacher, country, english, feels

In [5]:
#trying NMF with TF-IDF on original text with bigrams
docs = corpus_df.TEXT
tf = TfidfVectorizer(stop_words='english', min_df = 0.05, max_df=0.60, ngram_range=(1,2))
doc_term = tf.fit_transform(docs)
nmf = NMF(7)
nmf.fit(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 20)

/Users/ami/opt/anaconda3/envs/metis/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  1
husband, parents, daughter, apartment, story, phone, book, son, class, married, city, evening, teacher, marriage, books, dinner, reading, train, spoke, girls

Topic  2
says, asks, doesn, looks, thinks, takes, feels, goes, tells, turns, comes, knows, sees, makes, sits, wants, puts, gets, starts, stands

Topic  3
james, doctor, ship, daughter, mary, son, motel, ain, willing, dog, mr, yeah, maid, book, feels, deck, captain, baby, storm, land

Topic  4
road, dog, trees, river, wind, sun, sky, rain, town, old man, boys, rose, green, sea, ran, grass, stone, ground, tree, watched

Topic  5
dad, mom, baby, guy, kids, kid, yeah, fucking, brother, phone, shit, fuck, jack, guys, hey, ass, sorry, nice, somebody, actually

Topic  6
mrs, mr, dr, bus, grace, girls, driver, monkey, mary, miss, hat, dad, husband, stolen, sir, town, grandfather, birthday, upstairs, grandmother

Topic  7
paul, baby, crystal, aunt, parents, boyfriend, ma, st, babies, daughter, lighter, asshole, letter, ha, mary,

In [293]:
#best topics

#NMF + TF-IDF + nouns only

docs = corpus_df.TEXT_NOUNS
tf = TfidfVectorizer(stop_words='english', min_df = 0.03, max_df=0.55)
doc_term = tf.fit_transform(docs)
nmf = NMF(n_components=6, init='nndsvda', max_iter=450)
nmf.fit(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 20)


Topic  1
[['apartment', 0.423], ['student', 0.314], ['class', 0.297], ['train', 0.272], ['office', 0.27], ['hotel', 0.268], ['bar', 0.263], ['painting', 0.255], ['building', 0.241], ['dinner', 0.241], ['writer', 0.238], ['coffee', 0.235], ['restaurant', 0.233], ['party', 0.233], ['city', 0.232], ['drink', 0.22], ['desk', 0.22], ['teacher', 0.22], ['movie', 0.217], ['film', 0.213]]

Topic  2
[['road', 0.46], ['river', 0.268], ['wind', 0.265], ['stone', 0.264], ['sun', 0.26], ['field', 0.258], ['sea', 0.252], ['horse', 0.24], ['grass', 0.23], ['sky', 0.226], ['wood', 0.215], ['beach', 0.214], ['boat', 0.204], ['rain', 0.199], ['ground', 0.196], ['truck', 0.195], ['lake', 0.193], ['bird', 0.191], ['rock', 0.191], ['mountain', 0.19]]

Topic  3
[['kid', 0.865], ['guy', 0.819], ['mom', 0.463], ['shit', 0.267], ['dad', 0.266], ['cop', 0.261], ['brother', 0.256], ['ass', 0.225], ['yard', 0.181], ['son', 0.179], ['gun', 0.169], ['movie', 0.161], ['fuck', 0.155], ['stuff', 0.144], ['hell', 0.13

In [115]:
#NMF + TF-IDF + nouns + adjs only
docs = corpus_df.TEXT_NOUNS_ADJS
tf = TfidfVectorizer(stop_words='english', min_df = 0.05, max_df=0.60)
doc_term = tf.fit_transform(docs)
nmf = NMF(n_components=5, init='nndsvda', max_iter=450)
nmf.fit(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 20)


Topic  1
parent, apartment, student, husband, class, phone, love, daughter, teacher, office, painting, dinner, city, evening, train, party, marriage, college, letter, hotel

Topic  2
road, wind, stone, sun, river, field, town, village, sea, horse, sky, grass, wood, rain, beach, ground, boat, green, truck, bird

Topic  3
kid, guy, mom, fucking, phone, shit, brother, dad, cop, ass, nice, movie, fuck, bag, stuff, yard, bar, son, gun, beer

Topic  4
baby, husband, sister, daughter, doctor, son, nurse, grandmother, hospital, parent, law, brother, pregnant, skin, diaper, phone, doll, blood, aunt, bag

Topic  5
dog, doctor, porch, gate, yard, animal, pet, mask, town, fear, road, bird, paw, tail, phone, fur, fence, killer, blood, shovel




In [169]:
#NMF + TF-IDF + nouns + verbs only
docs = corpus_df.TEXT_NOUNS_VERBS
tf = TfidfVectorizer(stop_words='english', min_df = 0.05, max_df=0.60)
doc_term = tf.fit_transform(docs)
nmf = NMF(n_components=5, init='nndsvda', max_iter=450)
nmf.fit(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 20)


Topic  1
[['parent', 0.394], ['student', 0.307], ['husband', 0.305], ['apartment', 0.303], ['class', 0.262], ['daughter', 0.26], ['teacher', 0.25], ['office', 0.224], ['marry', 0.216], ['marriage', 0.214], ['letter', 0.213], ['evening', 0.213], ['train', 0.211], ['painting', 0.211], ['city', 0.209], ['college', 0.204], ['dinner', 0.203], ['writer', 0.202], ['restaurant', 0.194], ['party', 0.194]]

Topic  2
[['road', 0.369], ['wind', 0.24], ['sun', 0.23], ['stone', 0.228], ['river', 0.219], ['sea', 0.21], ['sky', 0.209], ['field', 0.209], ['rain', 0.201], ['town', 0.198], ['rise', 0.19], ['grass', 0.188], ['horse', 0.182], ['wood', 0.179], ['village', 0.177], ['beach', 0.177], ['lift', 0.172], ['boat', 0.171], ['climb', 0.168], ['ground', 0.168]]

Topic  3
[['kid', 0.742], ['guy', 0.715], ['mom', 0.389], ['fuck', 0.33], ['shit', 0.269], ['brother', 0.264], ['dad', 0.232], ['cop', 0.223], ['ass', 0.213], ['hit', 0.191], ['movie', 0.187], ['kill', 0.165], ['bag', 0.155], ['stuff', 0.151]

In [268]:
#NMF + TF-IDF + nouns + verbs + adjs
docs = corpus_df.TEXT_NOUNS_VERBS_ADJS
tf = TfidfVectorizer(stop_words='english', min_df = 0.02, max_df=0.55)
doc_term = tf.fit_transform(docs) 
nmf = NMF(n_components=6, init='nndsvda', max_iter=450)
NMF_doc_topic = nmf.fit_transform(doc_term)

output = display_topics(nmf, tf.get_feature_names(), 25)


Topic  1
[['student', 0.298], ['husband', 0.275], ['daughter', 0.249], ['teacher', 0.24], ['class', 0.23], ['writer', 0.209], ['marriage', 0.197], ['son', 0.183], ['letter', 0.18], ['marry', 0.175], ['college', 0.171], ['novel', 0.168], ['painting', 0.164], ['city', 0.161], ['poem', 0.159], ['certain', 0.154], ['office', 0.154], ['conversation', 0.153], ['art', 0.153], ['study', 0.151], ['reply', 0.15], ['train', 0.149], ['death', 0.149], ['apartment', 0.146], ['wish', 0.145]]

Topic  2
[['bus', 0.23], ['box', 0.207], ['bottle', 0.201], ['bedroom', 0.198], ['tea', 0.191], ['coat', 0.191], ['kiss', 0.185], ['apartment', 0.182], ['bathroom', 0.18], ['coffee', 0.176], ['wine', 0.168], ['bar', 0.164], ['cat', 0.162], ['mirror', 0.162], ['plate', 0.157], ['rain', 0.157], ['sofa', 0.155], ['hall', 0.152], ['jacket', 0.147], ['paint', 0.145], ['desk', 0.143], ['lift', 0.141], ['towel', 0.141], ['shop', 0.14], ['shoe', 0.137]]

Topic  3
[['guy', 0.653], ['mom', 0.359], ['fuck', 0.331], ['brot

In [274]:
#looking at topics for each doc
NMF_doc_topic = nmf.fit_transform(doc_term)

NMF_doc_topic_df = pd.DataFrame(NMF_doc_topic.round(3),
                 columns = [ix for ix, val in enumerate(nmf.components_)]
                )
NMF_doc_topic_df

,0,1,2,3,4,5
0,0.047,0.064,0.011,0.023,0.088,0.092
1,0.044,0.022,0.012,0.259,0.000,0.013
2,0.131,0.018,0.000,0.000,0.001,0.026
3,0.113,0.025,0.039,0.000,0.013,0.020
4,0.011,0.010,0.000,0.000,0.017,0.150
...,...,...,...,...,...,...
939,0.030,0.074,0.019,0.038,0.015,0.046
940,0.065,0.038,0.063,0.018,0.000,0.007
941,0.099,0.049,0.000,0.033,0.000,0.055
942,0.000,0.062,0.000,0.062,0.001,0.097


In [275]:
doc_term_df = pd.DataFrame(doc_term.toarray(), columns=tf.get_feature_names())
doc_term_df.head()

,ability,absence,abuse,accent,acceptance,access,accident,account,accountant,accusation,...,wrong,xa0,yard,yellow,yesterday,yogurt,youth,zipper,zone,zoo
0,0.041646,0.000000,0.0,0.018861,0.0,0.000000,0.0,0.000000,0.0,0.027148,...,0.0,0.0,0.026827,0.0,0.0,0.0,0.037480,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.090974,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.044209,0.0,0.000000,0.0,0.000000,0.0,0.083967,0.0,0.000000,...,0.0,0.0,0.030687,0.0,0.0,0.0,0.042873,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.000000,0.0,0.053752,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.060354,0.0,0.0,0.0,0.042160,0.0,0.0,0.0


In [280]:
(doc_term_df.loc[213].sort_values(ascending=False)).head(20)

terrace         0.340295
husband         0.340072
son             0.338806
grandson        0.305965
daughter        0.303975
doll            0.285321
baby            0.198757
care            0.119805
toast           0.112181
birth           0.104781
pool            0.104750
church          0.103857
doctor          0.103636
oil             0.098811
devil           0.088858
champagne       0.088405
bath            0.084211
sofa            0.075651
neighborhood    0.071311
apartment       0.070222
Name: 213, dtype: float64

In [277]:
#looking at top doc for each topic
for ix, val in enumerate(nmf.components_):
    max_story = np.argmax(NMF_doc_topic_df[ix])
    print('Topic', ix+1, max_story, corpus_df.TITLE[max_story], corpus_df.AUTHOR[max_story])

Topic 1 30 UNREAD MESSAGES SALLY ROONEY
Topic 2 490 TRANSATLANTIC COLUM MCCANN
Topic 3 22 THE MOM OF BOLD ACTION GEORGE SAUNDERS
Topic 4 838 MIRACLE JUDY BUDNITZ
Topic 5 555 THE YELLOW SAMANTHA HUNT
Topic 6 213 SUNRISE, SUNSET EDWIDGE DANTICAT


In [254]:
corpus_df.index

RangeIndex(start=0, stop=944, step=1)

In [202]:
nmf.components_

array([[4.77858167e-02, 5.85375697e-03, 3.01134251e-03, ...,
        3.76326404e-03, 0.00000000e+00, 1.05751304e-02],
       [3.86466420e-02, 2.28827264e-03, 7.18930569e-03, ...,
        8.32111881e-03, 9.74323661e-03, 0.00000000e+00],
       [0.00000000e+00, 1.81671932e-03, 3.75291669e-05, ...,
        3.19516403e-02, 9.46881748e-03, 0.00000000e+00],
       [1.34340730e-02, 5.96746123e-03, 9.22386291e-03, ...,
        0.00000000e+00, 2.87326217e-03, 6.29028164e-04],
       [1.95109278e-02, 1.76693754e-03, 0.00000000e+00, ...,
        1.78346888e-02, 3.40439740e-03, 0.00000000e+00]])

In [237]:
corpus_df[corpus_df.TITLE =='TRANSATLANTIC']

,URL,TITLE,AUTHOR,DATE,TEXT,WORDCOUNT,TEXT_NOUNS,TEXT_NOUNS_ADJS,TEXT_NOUNS_VERBS,TEXT_NOUNS_VERBS_ADJS,TEXT_ENTS
491,https://www.newyorker.com/magazine/2012/04/16/transatlantic-3,TRANSATLANTIC,COLUM MCCANN,"April 9, 2012","['The Vickers Vimy. A modified bomber. All wood and linen and wire. She’s wide and lumbering, but Jack Alcock still thinks her a nippy little thing. He pats her each time he climbs on board and slides into the cockpit beside Teddy Brown. One smoo...",8269,bomber wood linen wire thing time board cockpit motion body throttle foot rudder bar cloud sunlight edge shadow whiteness contracting surface cloud navigator fuss cockpit machine shape wind faith compass chart spirit level foot time century idea ...,bomber wood linen wire wide lumbering nippy little thing time board cockpit smooth motion body throttle foot rudder bar cloud sunlight edge shadow whiteness contracting surface cloud navigator reserved fuss cockpit alert machine shape wind faith ...,modify bomber wood linen wire think thing pat time climb board slide cockpit motion body hand throttle foot rudder bar feel like rise cloud fly sunlight lean edge shadow play whiteness expand contracting surface cloud navigator embarrass fuss si...,modify bomber wood linen wire wide lumbering think nippy little thing pat time climb board slide cockpit smooth motion body hand throttle foot rudder bar feel like rise cloud fly sunlight lean edge shadow play whiteness expand contracting surfac...,"[Europe, Suvla Bay, Kedos, France, London, St. John’s, England, Paris, Rome, Edinburgh, Salzburg, Sydney, London, Atlantic, Manchester, Scotland, Manchester, London, Vimy, London, St. Louis, St. Louis, St. Louis, the St. John’s, Concepçion Bay, S..."


In [238]:
corpus_df[corpus_df.TITLE =='MISS LORA']

,URL,TITLE,AUTHOR,DATE,TEXT,WORDCOUNT,TEXT_NOUNS,TEXT_NOUNS_ADJS,TEXT_NOUNS_VERBS,TEXT_NOUNS_VERBS_ADJS,TEXT_ENTS
490,https://www.newyorker.com/magazine/2012/04/23/miss-lora,MISS LORA,JUNOT DIAZ,"April 16, 2012","['Years later, you would wonder if it hadn’t been for your brother would you have done it? You’d remember how all the other guys had hated on her—how skinny she was, no culo, no titties, como un palito, but your brother didn’t care. I’d fuck her....",5145,year brother guy culo titty palito brother eye sound thing brother cancer sadness asshole end month cab street green ex girlfriend convulsion panic impulse sight week mother word mother silence day word today head answer age love girl expression...,year brother guy skinny culo titty palito brother eye sound bad thing brother dead cancer sadness super asshole end month cab street green ex - girlfriend convulsion panic atavistic impulse sight week feeble mother single word mother stubborn sil...,year wonder brother remember guy hate culo titty palito brother care fuck fuck jeer eye sound thing brother cancer feel fulgurate sadness asshole end die month try run catch try hail cab walk street green con ex girlfriend drive start convulsio...,year wonder brother remember guy hate skinny culo titty palito brother care fuck fuck jeer eye sound bad thing brother dead cancer feel fulgurate sadness super asshole end die month try run catch try hail cab walk street green con ex girlfriend...,"[Beth Israel, Newark, California, Camden, Paloma, Paloma, Poconos, Panama, Sayreville, La Vega, Paloma, Paloma, Paloma, Panama, N.Y.C., Chicken Holiday, Caribbean, Paloma, Paloma, New York City, Santo Domingo, Rome, Ann Arbor, Michigan, Germany, ..."


In [242]:
corpus_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 944 entries, 0 to 950
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   URL                    944 non-null    object
 1   TITLE                  944 non-null    object
 2   AUTHOR                 944 non-null    object
 3   DATE                   944 non-null    object
 4   TEXT                   944 non-null    object
 5   WORDCOUNT              944 non-null    int64 
 6   TEXT_NOUNS             944 non-null    object
 7   TEXT_NOUNS_ADJS        944 non-null    object
 8   TEXT_NOUNS_VERBS       944 non-null    object
 9   TEXT_NOUNS_VERBS_ADJS  944 non-null    object
 10  TEXT_ENTS              944 non-null    object
dtypes: int64(1), object(10)
memory usage: 120.8+ KB


In [243]:
NMF_doc_topic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       944 non-null    float64
 1   1       944 non-null    float64
 2   2       944 non-null    float64
 3   3       944 non-null    float64
 4   4       944 non-null    float64
dtypes: float64(5)
memory usage: 37.0 KB


In [240]:
NMF_doc_topic_df.loc[490]

0    0.103
1    0.025
2    0.053
3    0.000
4    0.024
Name: 491, dtype: float64

In [220]:
NMF_doc_topic_df.sort_values(1, ascending=False)

,0,1,2,3,4
490,0.000,0.218,0.000,0.000,0.000
579,0.000,0.198,0.000,0.062,0.038
19,0.012,0.191,0.001,0.017,0.028
667,0.000,0.188,0.022,0.029,0.019
874,0.000,0.187,0.000,0.000,0.000
...,...,...,...,...,...
719,0.154,0.000,0.000,0.000,0.005
823,0.008,0.000,0.116,0.032,0.008
477,0.026,0.000,0.000,0.143,0.000
580,0.004,0.000,0.026,0.000,0.386


In [281]:
#pickle
NMF_doc_topic_df.to_pickle("NMF_doc_topic_df3.pkl")

## LSA 
***

In [163]:
#LSA with TF-IDF
#looking at nouns only
docs = corpus_df.TEXT_NOUNS
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.55)
doc_term = tf.fit_transform(docs)
lsa = TruncatedSVD(5)
lsa.fit(doc_term)
topic_term = lsa.components_
output = display_topics(model=lsa, feature_names=tf.get_feature_names(), no_top_words=20)


Topic  1
[['dog', 0.109], ['husband', 0.107], ['son', 0.107], ['kid', 0.107], ['baby', 0.105], ['guy', 0.098], ['brother', 0.093], ['daughter', 0.092], ['apartment', 0.089], ['sister', 0.085], ['road', 0.08], ['doctor', 0.077], ['bar', 0.07], ['city', 0.07], ['dinner', 0.069], ['coffee', 0.067], ['office', 0.064], ['box', 0.063], ['bus', 0.063], ['party', 0.062]]

Topic  2
[['dog', 0.391], ['road', 0.192], ['river', 0.109], ['gun', 0.103], ['horse', 0.101], ['yard', 0.1], ['field', 0.094], ['truck', 0.088], ['rock', 0.088], ['wind', 0.084], ['sun', 0.084], ['kid', 0.082], ['grass', 0.082], ['wood', 0.08], ['bird', 0.076], ['fence', 0.072], ['cabin', 0.071], ['stone', 0.071], ['porch', 0.069], ['cow', 0.067]]

Topic  3
[['baby', 0.419], ['kid', 0.353], ['guy', 0.295], ['mom', 0.205], ['dog', 0.163], ['dad', 0.101], ['cop', 0.099], ['shit', 0.099], ['ass', 0.089], ['doctor', 0.089], ['sister', 0.086], ['nurse', 0.077], ['apartment', 0.068], ['couch', 0.065], ['movie', 0.064], ['dollar',

In [23]:
#nouns plus adjectives
docs = corpus_df.TEXT_NOUNS_ADJS
tf = TfidfVectorizer(stop_words='english', max_df=0.55)
doc_term = tf.fit_transform(docs)
lsa = TruncatedSVD(7)
lsa.fit(doc_term)
topic_term = lsa.components_
output = display_topics(model=lsa, feature_names=tf.get_feature_names(), no_top_words=20)


Topic  1
dog, baby, husband, kid, son, brother, guy, apartment, daughter, sister, road, doctor, city, bar, dinner, coffee, bus, box, office, bottle

Topic  2
dog, road, river, yard, horse, gun, rock, kid, field, sun, truck, wind, cabin, wood, porch, grass, stone, boat, fish, fence

Topic  3
baby, kid, guy, mom, dog, cop, shit, apartment, dad, fucking, ass, doctor, brother, nurse, movie, couch, son, husband, nice, dollar

Topic  4
baby, sister, husband, grandmother, son, village, daughter, nurse, doctor, grandfather, hospital, law, uncle, aunt, brother, church, doll, cousin, stroller, dress

Topic  5
dog, husband, daughter, doctor, baby, nurse, apartment, tea, hospital, gate, painting, wedding, leash, film, piano, grandmother, garden, sister, painter, mask

Topic  6
cat, bar, apartment, drink, bottle, bathroom, wine, bedroom, sofa, baby, box, mirror, towel, coat, train, coffee, hall, hotel, sink, party

Topic  7
baby, poem, writer, sea, fish, film, beach, human, novel, boat, poet, rock

In [24]:
#trying nouns and verbs
docs = corpus_df.TEXT_NOUNS_VERBS
tf = TfidfVectorizer(stop_words='english', max_df=0.55)
doc_term = tf.fit_transform(docs)
lsa = TruncatedSVD(7)
lsa.fit(doc_term)
topic_term = lsa.components_
output = display_topics(model=lsa, feature_names=tf.get_feature_names(), no_top_words=20)


Topic  1
dog, husband, baby, son, brother, guy, daughter, apartment, sister, road, doctor, bar, city, dinner, coffee, kiss, bus, box, dream, office

Topic  2
dog, road, river, gun, yard, sun, rock, horse, truck, field, fish, wind, rain, wood, cabin, grass, boat, porch, stone, fence

Topic  3
baby, guy, mom, husband, apartment, doctor, nurse, sister, daughter, grandmother, fuck, son, shit, hospital, dad, couch, brother, cop, ass, dollar

Topic  4
dog, guy, movie, apartment, painting, film, bar, fuck, mail, writer, poem, cop, hotel, painter, building, text, mom, shit, tv, restaurant

Topic  5
dog, husband, daughter, doctor, tea, baby, sister, garden, grandmother, gate, village, nurse, wedding, marry, piano, hospital, cottage, leash, teacher, marriage

Topic  6
baby, apartment, cat, bottle, bathroom, painting, wine, bedroom, sofa, mirror, kiss, coat, towel, box, coffee, bar, bus, hall, plate, rain

Topic  7
baby, poem, doctor, nurse, film, sea, writer, poet, beach, painting, river, rock,

In [25]:
#LSA with nouns, verbs, adjectives
docs = corpus_df.TEXT_NOUNS_VERBS_ADJS
tf = TfidfVectorizer(stop_words='english', max_df=0.55)
doc_term = tf.fit_transform(docs)
lsa = TruncatedSVD(7)
lsa.fit(doc_term)
topic_term = lsa.components_
output = display_topics(model=lsa, feature_names=tf.get_feature_names(), no_top_words=20)


Topic  1
dog, baby, husband, son, brother, guy, apartment, daughter, sister, road, doctor, bar, city, dinner, coffee, kiss, bus, box, dream, office

Topic  2
dog, road, river, gun, sun, yard, rock, truck, wind, horse, field, rain, cabin, wood, fish, boat, baby, grass, stone, porch

Topic  3
baby, guy, mom, apartment, fuck, shit, fucking, brother, cop, dad, ass, doctor, nurse, husband, couch, sister, movie, son, dollar, ok

Topic  4
baby, husband, sister, grandmother, daughter, village, son, nurse, doctor, hospital, grandfather, law, tea, doll, aunt, garden, uncle, church, marry, milk

Topic  5
dog, husband, daughter, doctor, tea, gate, leash, film, pet, wedding, fuckin, porch, marry, mask, coat, garden, piano, wine, painting, drawer

Topic  6
painting, apartment, kiss, bottle, bathroom, sofa, wine, mirror, bedroom, bar, towel, coat, coffee, paint, box, jacket, hotel, bus, plate, balcony

Topic  7
baby, poem, fish, writer, film, human, sea, movie, nurse, poet, doctor, beach, stroller, 

In [137]:
doc_topic = lsa.fit_transform(doc_term)

doc_topic_df = pd.DataFrame(doc_topic.round(3),
                 index = corpus_df.TITLE,
                 columns = [ix for ix, val in enumerate(lsa.components_)]
                )
doc_topic_df

,0,1,2,3,4
TITLE,,,,,
ANNUNCIATION,0.424,0.027,-0.020,0.090,0.136
ONCE REMOVED,0.396,-0.036,0.178,0.204,-0.069
LONG DISTANCE,0.348,-0.137,-0.102,-0.074,0.068
"WHAT'S THE DEAL, HUMMINGBIRD?",0.384,-0.097,-0.039,-0.124,0.016
FIREWORKS,0.223,-0.087,-0.035,0.150,0.074
...,...,...,...,...,...
THE GUARDIANS,0.318,0.039,-0.032,0.074,-0.020
U.F.O. IN KUSHIRO,0.330,-0.012,0.024,-0.060,-0.050
NOBODY'S BUSINESS,0.403,-0.079,-0.047,0.031,0.023


In [141]:
#looking at top doc for each topic
for ix, val in enumerate(lsa.components_):
    max_story = np.argmax(doc_topic_df[ix], axis=-1)
    print(max_story, corpus_df.TITLE[max_story], corpus_df.AUTHOR[max_story], np.max(val))

662 DON'T CRY MARY GAITSKILL 0.10855776055297554
516 THE HOUSE ON SAND CREEK THOMAS MCGUANE 0.39268225403261736
529 ABOVE AND BELOW LAUREN GROFF 0.4248772732555857
394 THE RELIVE BOX T. CORAGHESSAN BOYLE 0.4246200167870725
555 COSTELLO JIM GAVIN 0.7738145742377834


## LDA
***

In [143]:
from gensim import corpora, models, matutils

#looking at nouns only
docs = corpus_df.TEXT_NOUNS
vectorizer = CountVectorizer(stop_words='english', max_df=0.55)
term_doc_matrix = vectorizer.fit_transform(docs).transpose()

corpus = matutils.Sparse2Corpus(term_doc_matrix)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=7, id2word=id2word, passes=150)

lda.print_topics()

[(0,
  '0.010*"village" + 0.007*"road" + 0.006*"kid" + 0.004*"field" + 0.004*"ground" + 0.004*"sir" + 0.004*"hut" + 0.004*"brother" + 0.004*"gun" + 0.004*"cousin"'),
 (1,
  '0.012*"brother" + 0.007*"guy" + 0.006*"kid" + 0.006*"son" + 0.005*"monkey" + 0.005*"movie" + 0.005*"bar" + 0.005*"sister" + 0.004*"beer" + 0.004*"apartment"'),
 (2,
  '0.012*"baby" + 0.012*"sister" + 0.011*"doctor" + 0.009*"husband" + 0.008*"daughter" + 0.007*"hospital" + 0.007*"kid" + 0.007*"grandmother" + 0.006*"law" + 0.006*"nurse"'),
 (3,
  '0.008*"dog" + 0.005*"road" + 0.004*"cat" + 0.004*"sun" + 0.004*"yard" + 0.003*"wind" + 0.003*"sky" + 0.003*"kid" + 0.003*"fish" + 0.003*"bottle"'),
 (4,
  '0.005*"apartment" + 0.005*"son" + 0.005*"guy" + 0.004*"student" + 0.004*"class" + 0.003*"office" + 0.003*"daughter" + 0.003*"kid" + 0.003*"painting" + 0.003*"college"'),
 (5,
  '0.010*"baby" + 0.004*"kid" + 0.004*"police" + 0.004*"box" + 0.004*"desk" + 0.004*"bus" + 0.004*"son" + 0.003*"blood" + 0.003*"policeman" + 0.003

In [32]:
#nouns and verbs
docs = corpus_df.TEXT_NOUNS_VERBS
vectorizer = CountVectorizer(stop_words='english', min_df = 0.05, max_df=0.60)
term_doc_matrix = vectorizer.fit_transform(docs).transpose()

corpus = matutils.Sparse2Corpus(term_doc_matrix)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=6, id2word=id2word, passes=250)

lda.print_topics()

[(0,
  '0.007*"road" + 0.006*"sun" + 0.006*"sky" + 0.005*"sea" + 0.005*"wave" + 0.005*"wind" + 0.005*"rise" + 0.005*"fish" + 0.005*"beach" + 0.004*"cat"'),
 (1,
  '0.006*"parent" + 0.005*"husband" + 0.004*"daughter" + 0.004*"student" + 0.004*"class" + 0.003*"evening" + 0.003*"teacher" + 0.003*"city" + 0.003*"office" + 0.003*"conversation"'),
 (2,
  '0.010*"son" + 0.009*"brother" + 0.007*"kill" + 0.007*"road" + 0.007*"gun" + 0.006*"town" + 0.006*"village" + 0.006*"church" + 0.005*"police" + 0.005*"ground"'),
 (3,
  '0.007*"tea" + 0.005*"paint" + 0.005*"town" + 0.005*"bus" + 0.005*"shop" + 0.005*"grandmother" + 0.005*"evening" + 0.005*"clothe" + 0.005*"box" + 0.005*"grandfather"'),
 (4,
  '0.017*"dog" + 0.015*"baby" + 0.010*"sister" + 0.008*"doctor" + 0.007*"daughter" + 0.005*"nurse" + 0.005*"blood" + 0.005*"bird" + 0.005*"dream" + 0.004*"husband"'),
 (5,
  '0.014*"kid" + 0.011*"guy" + 0.005*"apartment" + 0.005*"bag" + 0.005*"mom" + 0.005*"fuck" + 0.004*"brother" + 0.004*"check" + 0.004*

In [33]:
#nouns and verbs and adjs

from gensim import corpora, models, matutils

docs = corpus_df.TEXT_NOUNS_VERBS_ADJS
vectorizer = CountVectorizer(stop_words='english', min_df = 10, max_df=0.60)
term_doc_matrix = vectorizer.fit_transform(docs).transpose()

corpus = matutils.Sparse2Corpus(term_doc_matrix)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=6, id2word=id2word, passes=350)

lda.print_topics()

[(0,
  '0.011*"husband" + 0.009*"daughter" + 0.008*"doctor" + 0.007*"parent" + 0.006*"baby" + 0.006*"son" + 0.006*"sister" + 0.005*"hospital" + 0.004*"brother" + 0.004*"nurse"'),
 (1,
  '0.006*"kid" + 0.004*"dog" + 0.004*"guy" + 0.003*"bag" + 0.003*"baby" + 0.003*"shirt" + 0.003*"road" + 0.003*"roll" + 0.003*"bottle" + 0.002*"seat"'),
 (2,
  '0.007*"son" + 0.004*"village" + 0.004*"brother" + 0.004*"church" + 0.004*"town" + 0.004*"death" + 0.004*"grandfather" + 0.003*"parent" + 0.003*"kill" + 0.003*"road"'),
 (3,
  '0.004*"student" + 0.004*"class" + 0.004*"teacher" + 0.004*"evening" + 0.003*"train" + 0.003*"town" + 0.003*"parent" + 0.003*"summer" + 0.003*"hotel" + 0.002*"bus"'),
 (4,
  '0.004*"sky" + 0.004*"stone" + 0.003*"wind" + 0.003*"sea" + 0.003*"rise" + 0.003*"sun" + 0.003*"green" + 0.003*"road" + 0.003*"wave" + 0.003*"bird"'),
 (5,
  '0.004*"guy" + 0.004*"apartment" + 0.003*"movie" + 0.003*"painting" + 0.002*"bar" + 0.002*"kid" + 0.002*"building" + 0.002*"self" + 0.002*"certain" 

In [46]:
#try with TFIDF?
docs = corpus_df.TEXT_NOUNS_VERBS_ADJS
vectorizer = TfidfVectorizer(stop_words='english', min_df = .05, max_df=0.60)
term_doc_matrix = vectorizer.fit_transform(docs).transpose()

corpus = matutils.Sparse2Corpus(term_doc_matrix)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=6, id2word=id2word, passes=350)

lda.print_topics()

[(0,
  '0.001*"dog" + 0.001*"kid" + 0.001*"husband" + 0.001*"son" + 0.001*"baby" + 0.001*"parent" + 0.001*"guy" + 0.001*"brother" + 0.001*"daughter" + 0.001*"apartment"'),
 (1,
  '0.000*"colonize" + 0.000*"valid" + 0.000*"refund" + 0.000*"mindedness" + 0.000*"whirlwind" + 0.000*"fig" + 0.000*"cuss" + 0.000*"chrysanthemum" + 0.000*"unfounded" + 0.000*"facilitate"'),
 (2,
  '0.000*"colonize" + 0.000*"valid" + 0.000*"refund" + 0.000*"mindedness" + 0.000*"whirlwind" + 0.000*"fig" + 0.000*"cuss" + 0.000*"chrysanthemum" + 0.000*"unfounded" + 0.000*"facilitate"'),
 (3,
  '0.000*"colonize" + 0.000*"valid" + 0.000*"refund" + 0.000*"mindedness" + 0.000*"whirlwind" + 0.000*"fig" + 0.000*"cuss" + 0.000*"chrysanthemum" + 0.000*"unfounded" + 0.000*"facilitate"'),
 (4,
  '0.000*"colonize" + 0.000*"valid" + 0.000*"refund" + 0.000*"mindedness" + 0.000*"whirlwind" + 0.000*"fig" + 0.000*"cuss" + 0.000*"chrysanthemum" + 0.000*"unfounded" + 0.000*"facilitate"'),
 (5,
  '0.000*"colonize" + 0.000*"valid" + 0

In [259]:
# create dataframe showing topic probability scores for each document

from collections import defaultdict

def lda_df_maker(data, n_topics):
    topic_doc_dist = defaultdict(list)
    for ix, val in enumerate(data):
        doc_top_list = lda.get_document_topics(corpus[ix], minimum_probability=0, minimum_phi_value=0)
        for pair in doc_top_list:
            topic_num = pair[0]
            topic_score = pair[1]
            topic_doc_dist[topic_num].append(np.round(topic_score, 3))
    return topic_doc_dist

lda.get_document_topics(corpus[2], minimum_probability=0, minimum_phi_value=0)

[(0, 0.13469422),
 (1, 0.051296018),
 (2, 0.043109056),
 (3, 0.0002083209),
 (4, 0.25231516),
 (5, 0.4360926),
 (6, 0.08228463)]

In [260]:
lda_df_dict = lda_df_maker(corpus, n_topics=4)
lda_df_dict[0:5]

defaultdict(list,
            {0: [0.0,
              0.001,
              0.135,
              0.04,
              0.001,
              0.0,
              0.0,
              0.103,
              0.0,
              0.064,
              0.355,
              0.0,
              0.007,
              0.169,
              0.0,
              0.0,
              0.167,
              0.366,
              0.0,
              0.09,
              0.0,
              0.035,
              0.064,
              0.0,
              0.001,
              0.063,
              0.255,
              0.0,
              0.138,
              0.0,
              0.032,
              0.115,
              0.018,
              0.01,
              0.072,
              0.037,
              0.257,
              0.142,
              0.0,
              0.0,
              0.0,
              0.0,
              0.791,
              0.015,
              0.0,
              0.001,
              0.079,
              0.138,
        

In [261]:
lda_df = pd.DataFrame(lda_df_dict)
lda_df

,0,1,2,3,4,5,6
0,0.000,0.068,0.281,0.101,0.027,0.000,0.523
1,0.001,0.317,0.001,0.001,0.086,0.142,0.454
2,0.135,0.051,0.043,0.000,0.252,0.436,0.082
3,0.040,0.112,0.355,0.217,0.027,0.249,0.000
4,0.001,0.183,0.033,0.285,0.039,0.213,0.247
...,...,...,...,...,...,...,...
941,0.077,0.243,0.000,0.185,0.090,0.187,0.217
942,0.311,0.000,0.065,0.183,0.062,0.009,0.370
943,0.001,0.228,0.148,0.493,0.001,0.001,0.128
944,0.011,0.115,0.157,0.337,0.136,0.244,0.000


In [47]:
dominant_topic = [np.argmax(lda_df.values, axis=1)]
lda_df['dominant_topic'] = dominant_topic
lda_df

NameError: name 'lda_df' is not defined

In [262]:
lda_df.insert(0, 'TITLE', corpus_df.TITLE, allow_duplicates=False)
lda_df.insert(1, 'AUTHOR', corpus_df.AUTHOR, allow_duplicates=True)
lda_df

,TITLE,AUTHOR,0,1,2,3,4,5,6
0,Once Removed,Alexander MacLeod,0.000,0.068,0.281,0.101,0.027,0.000,0.523
1,Fireworks,Graham Swift,0.001,0.317,0.001,0.001,0.086,0.142,0.454
2,What the Forest Remembers,Jennifer Egan,0.135,0.051,0.043,0.000,0.252,0.436,0.082
3,A Lot of Things Have Happened,Adam Levin,0.040,0.112,0.355,0.217,0.027,0.249,0.000
4,"Lu, Reshaping",Madeleine Thien,0.001,0.183,0.033,0.285,0.039,0.213,0.247
...,...,...,...,...,...,...,...,...,...
941,What Is Remembered,Alice Munro,0.077,0.243,0.000,0.185,0.090,0.187,0.217
942,My Widow,T. Coraghessan Boyle,0.311,0.000,0.065,0.183,0.062,0.009,0.370
943,Going for a Beer,Robert Coover,0.001,0.228,0.148,0.493,0.001,0.001,0.128
944,"What’s the Deal, Hummingbird?",Arthur Krystal,0.011,0.115,0.157,0.337,0.136,0.244,0.000


In [268]:
#7 topics, nouns and verbs
lda_max_story1 = np.argmax(lda_df[0])
lda_max_story2 = np.argmax(lda_df[1])
lda_max_story3 = np.argmax(lda_df[2])
lda_max_story4 = np.argmax(lda_df[3])
lda_max_story5 = np.argmax(lda_df[4])
lda_max_story6 = np.argmax(lda_df[5])
lda_max_story7 = np.argmax(lda_df[6])

print(f"Topic 1: {lda_df.iloc[lda_max_story1].TITLE, lda_df.iloc[lda_max_story1].AUTHOR, lda_df.iloc[lda_max_story1][0]} \
      \nTopic 2: {lda_df.iloc[lda_max_story2].TITLE, lda_df.iloc[lda_max_story2].AUTHOR, lda_df.iloc[lda_max_story2][1]} \
      \nTopic 3: {lda_df.iloc[lda_max_story3].TITLE, lda_df.iloc[lda_max_story3].AUTHOR, lda_df.iloc[lda_max_story3][2]} \
      \nTopic 4: {lda_df.iloc[lda_max_story4].TITLE, lda_df.iloc[lda_max_story4].AUTHOR, lda_df.iloc[lda_max_story4][3]} \
      \nTopic 5: {lda_df.iloc[lda_max_story5].TITLE, lda_df.iloc[lda_max_story5].AUTHOR, lda_df.iloc[lda_max_story5][4]} \
      \nTopic 6: {lda_df.iloc[lda_max_story6].TITLE, lda_df.iloc[lda_max_story6].AUTHOR, lda_df.iloc[lda_max_story6][5]} \
      \nTopic 7: {lda_df.iloc[lda_max_story7].TITLE, lda_df.iloc[lda_max_story7].AUTHOR, lda_df.iloc[lda_max_story7][6]}")

Topic 1: ('Cattle Praise Song', 'Scholastique Mukasonga', 0.9290000200271606)       
Topic 2: ('The Very Rigid Search', 'Jonathan Safran Foer', 0.824999988079071)       
Topic 3: ('Fly Already', 'Etgar Keret', 0.8809999823570251)       
Topic 4: ('A for Alone', 'Curtis Sittenfeld', 0.8389999866485596)       
Topic 5: ('Getting Closer', 'Steven Millhauser', 0.996999979019165)       
Topic 6: ('My Internet', 'Jonathan Lethem', 0.9810000061988831)       
Topic 7: ('Broccoli', 'Lara Vapnyar', 0.8880000114440918)


In [276]:
#6 topics, nouns and verbs and adjectives
lda_max_story1 = np.argmax(lda_df[0])
lda_max_story2 = np.argmax(lda_df[1])
lda_max_story3 = np.argmax(lda_df[2])
lda_max_story4 = np.argmax(lda_df[3])
lda_max_story5 = np.argmax(lda_df[4])
lda_max_story6 = np.argmax(lda_df[5])
lda_max_story7 = np.argmax(lda_df[6])

print(f"Topic 1: {lda_df.iloc[lda_max_story1].TITLE, lda_df.iloc[lda_max_story1].AUTHOR, lda_df.iloc[lda_max_story1][0]} \
      \nTopic 2: {lda_df.iloc[lda_max_story2].TITLE, lda_df.iloc[lda_max_story2].AUTHOR, lda_df.iloc[lda_max_story2][1]} \
      \nTopic 3: {lda_df.iloc[lda_max_story3].TITLE, lda_df.iloc[lda_max_story3].AUTHOR, lda_df.iloc[lda_max_story3][2]} \
      \nTopic 4: {lda_df.iloc[lda_max_story4].TITLE, lda_df.iloc[lda_max_story4].AUTHOR, lda_df.iloc[lda_max_story4][3]} \
      \nTopic 5: {lda_df.iloc[lda_max_story5].TITLE, lda_df.iloc[lda_max_story5].AUTHOR, lda_df.iloc[lda_max_story5][4]} \
      \nTopic 6: {lda_df.iloc[lda_max_story6].TITLE, lda_df.iloc[lda_max_story6].AUTHOR, lda_df.iloc[lda_max_story6][5]} \
      \nTopic 7: {lda_df.iloc[lda_max_story7].TITLE, lda_df.iloc[lda_max_story7].AUTHOR, lda_df.iloc[lda_max_story7][6]}")

Topic 1: ('Cattle Praise Song', 'Scholastique Mukasonga', 0.9290000200271606)       
Topic 2: ('The Very Rigid Search', 'Jonathan Safran Foer', 0.824999988079071)       
Topic 3: ('Fly Already', 'Etgar Keret', 0.8809999823570251)       
Topic 4: ('A for Alone', 'Curtis Sittenfeld', 0.8389999866485596)       
Topic 5: ('Getting Closer', 'Steven Millhauser', 0.996999979019165)       
Topic 6: ('My Internet', 'Jonathan Lethem', 0.9810000061988831)       
Topic 7: ('Broccoli', 'Lara Vapnyar', 0.8880000114440918)


In [249]:
#looking at sally rooney, unread messages
lda.get_document_topics(corpus[27])

[(1, 0.23249243), (2, 0.6100219), (4, 0.15263024)]

In [144]:
corpus_df[corpus_df['TITLE']=='The Semplica-Girl Diaries']

,TITLE,AUTHOR,DATE,TEXT,TEXT_STRIPPED,TEXT_WORDS,WORDCOUNT,TEXT_NOSTOP,TEXT_LEMMED
462,The Semplica-Girl Diaries,George Saunders,"October 8, 2012","Having just turned forty, have resolved to embark on grand project of writing every day in this new black book just got at OfficeMax. Exciting to ...",having just turned forty have resolved to embark on grand project of writing every day in this new black book just got at officemax exciting to th...,"[having, just, turned, forty, have, resolved, to, embark, on, grand, project, of, writing, every, day, in, this, new, black, book, just, got, at, ...",8875,"[having, turned, resolved, embark, grand, project, writing, day, new, black, book, got, officemax, exciting, think, year, rate, page/day, written,...",have turn resolve embark grand project write day new black book get officemax exciting year rate page / day write sixty - five page picture life a...


In [250]:
lda.get_document_topics(corpus[462])

[(0, 0.226393),
 (1, 0.22917193),
 (2, 0.033722848),
 (3, 0.22798003),
 (4, 0.16889687),
 (5, 0.013160927),
 (6, 0.10067436)]

In [157]:
corpus_df[corpus_df['TITLE']=='Cat Person']

,TITLE,AUTHOR,DATE,TEXT,TEXT_STRIPPED,TEXT_WORDS,WORDCOUNT,TEXT_NOSTOP,TEXT_LEMMED,TEXT_ENTS
199,Cat Person,Kristen Roupenian,"December 4, 2017",Margot met Robert on a Wednesday night toward the end of her fall semester. She was working behind the concession stand at the artsy movie theatre...,margot met robert on a wednesday night toward the end of her fall semester she was working behind the concession stand at the artsy movie theatre ...,"[margot, met, robert, on, a, wednesday, night, toward, the, end, of, her, fall, semester, she, was, working, behind, the, concession, stand, at, t...",7171,"[margot, met, robert, wednesday, night, end, fall, semester, working, concession, stand, artsy, movie, theatre, downtown, came, bought, large, pop...",margot meet robert wednesday night end fall semester work concession stand artsy movie theatre downtown buy large popcorn box red vine that s an ...,"[robert, wednesday night end fall semester, week, morning, expect minute, robert, ta freshman year, twenty - one, one, summer, robert, spring, rob..."


In [184]:
lda.get_document_topics(corpus[199])

[(0, 0.21020702), (3, 0.78709996)]

In [185]:
# Create Document — Topic Matrix
lda_output = lda.get_document_topics(corpus)

# Make the pandas dataframe
df_document_topic = pd.DataFrame(lda_output, index=corpus_df.TITLE)

df_document_topic

,0,1,2,3,4,5,6
TITLE,,,,,,,
Once Removed,"(0, 0.11989243)","(1, 0.030073835)","(5, 0.021415913)","(6, 0.8223274)",None,None,None
Fireworks,"(4, 0.7580817)","(5, 0.23122805)",None,None,None,None,None
What the Forest Remembers,"(0, 0.0719902)","(1, 0.14159422)","(3, 0.034919653)","(4, 0.08900859)","(5, 0.6374673)","(6, 0.016240016)",None
A Lot of Things Have Happened,"(0, 0.036858257)","(1, 0.5412843)","(2, 0.03783699)","(3, 0.06756999)","(4, 0.28427994)","(5, 0.017050512)","(6, 0.015119964)"
"Lu, Reshaping","(0, 0.13633803)","(2, 0.84390837)",None,None,None,None,None
...,...,...,...,...,...,...,...
What Is Remembered,"(0, 0.816487)","(3, 0.09551976)","(4, 0.0191965)","(5, 0.051472813)","(6, 0.015343459)",None,None
My Widow,"(0, 0.11592037)","(1, 0.020018876)","(2, 0.061765198)","(3, 0.01608048)","(4, 0.6756737)","(5, 0.0419466)","(6, 0.06859471)"
Going for a Beer,"(0, 0.24196984)","(2, 0.6079796)","(5, 0.14712824)",None,None,None,None


In [152]:
sp(corpus_df.TEXT[0]).similarity(sp(corpus_df.TEXT[462]))

0.9877527425559723

In [151]:
sp(corpus_df.TEXT_LEMMED[0]).similarity(sp(corpus_df.TEXT_LEMMED[462]))

0.9821807542368011

In [73]:
sp(corpus_df.TEXT_LEMMED[0]).similarity(sp(corpus_df.TEXT_LEMMED[27]))

0.9912693544576962

# CorEx

In [85]:
!pip install corextopic
!pip install networkx

In [105]:
from corextopic import corextopic as ct

docs = corpus_df.TEXT_LEMMED
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.60, ngram_range=(1,2))
tfidf = tf.fit_transform(docs)
vocab = tf.get_feature_names()

#anchors = []

model = ct.Corex(n_hidden=5, seed=42)
model = model.fit(tfidf, words=vocab)

topics = model.get_topics()
for topic_n,topic in enumerate(topics):
    # w: word, mi: mutual information, s: sign
    topic = [(w,mi,s) if s > 0 else ('~'+w,mi,s) for w,mi,s in topic]
    # Unpack the info about the topic
    words,mis,signs = zip(*topic)    
    # Print topic
    topic_str = str(topic_n+1)+': '+', '.join(words)
    print(topic_str)

1: internet, prince, eve, leader, frog, leaf, cure, lick, ~reputation, ~regard
2: pond, flight, suitcase, september, female, rustle, ~proud, ~property, ~prompt, ~ability
3: bible, angel, journal, wing, sort thing, brag, penis, incredible, ~prevent, ~principle
4: april, husband, ~academic, ~proceed, ~profound, ~promise, ~proof, ~propose, ~prospect, ~purpose
5: absence, presence, present, preserve, pretty, previous, previously, pride, probably nt, professional


In [104]:
docs = corpus_df.TEXT_NOUNS
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.60, ngram_range=(1,2))
tfidf = tf.fit_transform(docs)
vocab = tf.get_feature_names()

#anchors = []

model = ct.Corex(n_hidden=5, seed=42)
model = model.fit(tfidf, words=vocab)

topics = model.get_topics()
for topic_n,topic in enumerate(topics):
    # w: word, mi: mutual information, s: sign
    topic = [(w,mi,s) if s > 0 else ('~'+w,mi,s) for w,mi,s in topic]
    # Unpack the info about the topic
    words,mis,signs = zip(*topic)    
    # Print topic
    topic_str = str(topic_n+1)+': '+', '.join(words)
    print(topic_str)

1: internet, angel, wings, leader, angels, leaf, humans, apples, leaves, ~pots
2: ~absence, prayer, ~power, ~poster, ~portrait, ~politics, poison, ~playground, plants, ~planet
3: ~account, ~presence, prayers, potatoes, potato, posts, ~position, porch, pool, policeman
4: accent, ~property, product, practice, powers, powder, pound, ~pot, possibility, possibilities
5: ~actions, ~puzzle, ~punch, ~problems, ~prize, ~privacy, ~press, ~post, ~portion, ~pop


In [111]:

docs = corpus_df.TEXT_LEMMED
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.80, ngram_range=(1,2))
tfidf = tf.fit_transform(docs)
vocab = tf.get_feature_names()

anchors = [
    ["old", "age", "remember", "lifetime", "life", "death"],
    ["COVID", "pandemic", "virus", "mask", "vaccine", "plague"],
    ["attraction", "desire", "sex", "romance", "romantic", "girlfriend", "boyfriend", "wedding", "marriage"],
    ["paycheck", "bill", "rich", "estate", "interest rate", "debit", "credit"],
    ["English", "immigrate", "immigrant", "emigrate", "country"]
]

anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=8, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=4 # Tell the model how much it should rely on the anchors
)

topics = model.get_topics()
for topic_n,topic in enumerate(topics):
    # w: word, mi: mutual information, s: sign
    topic = [(w,mi,s) if s > 0 else ('~'+w,mi,s) for w,mi,s in topic]
    # Unpack the info about the topic
    words,mis,signs = zip(*topic)    
    # Print topic
    topic_str = str(topic_n+1)+': '+', '.join(words)
    print(topic_str)

1: accidentally, purple, push, push away, rage, raise hand, random, rank, reach, real life
2: mask, look mirror, avoid, flee, cross, walk home, mirror, ~receiver, ~recite, ~reflection
3: palace, miniature, invisible, master, king, visible, copper, task, apple, realm
4: george, ~absence, ~pour, ~precious, ~present, ~pressure, ~pride, ~priest, ~prisoner, ~profound
5: internet, leader, ~prop, ~proper, ~protrude, ~provoke, ~puddle, ~pupil, ~profession, ~absurd
6: prince, frog, pond, lick, mud, tongue, ~potato, ~power, ~praise, ~prevent
7: eve, leaf, cure, invent, ~prize, ~politician, ~pool, ~poorly, ~population, ~porch
8: crystal, paul, ~absent, ~refrigerator, relative, ~relatively, ~remind, ~renew, ~require, ~resemble


In [113]:
docs = corpus_df.TEXT_NOUNS
tf = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.80, ngram_range=(1,2))
tfidf = tf.fit_transform(docs)
vocab = tf.get_feature_names()

anchors = [
    ["age", "lifetime", "life", "death", "memory"],
    ["COVID", "pandemic", "virus", "plague"],
    ["attraction", "desire", "sex", "romance", "girlfriend", "boyfriend", "wedding", "marriage"],
    ["paycheck", "bill", "estate", "rate", "debit", "credit"],
    ["English", "immigrant", "country"], [], [], []
]

anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=8, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=4 # Tell the model how much it should rely on the anchors
)

topics = model.get_topics()
for topic_n,topic in enumerate(topics):
    # w: word, mi: mutual information, s: sign
    topic = [(w,mi,s) if s > 0 else ('~'+w,mi,s) for w,mi,s in topic]
    # Unpack the info about the topic
    words,mis,signs = zip(*topic)    
    # Print topic
    topic_str = str(topic_n+1)+': '+', '.join(words)
    print(topic_str)

1: account, ~rearview mirror, ~quarters, puzzle, protest, process, print, pride, pressure, pounds
2: accent, pictures, phone calls, pepper, people people, ~peace, ~patterns, patients, pillow, ~patience
3: internet, angel, wings, angels, leader, leaf, humans, apples, leaves, veins
4: ~accident, ~playing, ~play, ~platform, plant, ~plane, ~piano, ~photo, ~plot, ~personality
5: access, ~rim, ~revenge, ~resistance, report, remark, refrigerator, ~red, ~rat, ~range
6: ~accounts, radio, ~purpose, ~project, ~problem, ~prison, ~present, ~potato, ~poster, ~possibilities
7: ability, office, ~oil, ~operation, opinion, option, pad, patient, patio, phone
8: ~action, planes, ~plan, ~places, pink, pig, pieces, ~piece paper, ~phones, ~pencil


In [50]:
#try with countvectorizer
from corextopic import corextopic as ct
docs = corpus_df.TEXT_NOUNS_VERBS
vectorizer = CountVectorizer(max_features=20000,
                             stop_words='english', token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True, max_df=0.60)

doc_word = vectorizer.fit_transform(docs)
words = list(np.asarray(vectorizer.get_feature_names()))

anchors = [
    ["age", "lifetime", "life", "death", "memory", "die", "remember", "memorial", "widow"],
    ["COVID", "pandemic", "virus", "plague"],
    ["attraction", "desire", "sex", "romance", "girlfriend", "boyfriend", "wedding", "marriage", "kiss"],
    ["paycheck", "bill", "estate", "rate", "debit", "credit", "pay", "debt"],
    ["English", "immigrant", "country", "immigrate"]
]

anchors = [
    [a for a in topic if a in words]
    for topic in anchors
]


topic_model = ct.Corex(n_hidden=6, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=docs, anchors=anchors, anchor_strength=4)

topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: death,memory,roll,earth,neck,rise,nose,wrap,skin,gaze
1: plague,virus,pandemic,food,ass,blast,scan,guy,truck,pound
2: marriage,sex,romance,boyfriend,girlfriend,wedding,desire,kiss,self,evening
3: credit,rate,estate,debt,sign,office,result,site,project,account
4: country,immigrant,track,expect,lay,press,suggest,circle,style,card
5: hug,smear,pack,chance,lid,groan,pan,rinse,nudge,powder


In [51]:
topic_model.get_top_docs(topic=4, n_docs=2)[0:50]

[('girl catch cheat  fiancée bit matter catch sucia catch re cuero empty e  mail trash can catch year period girl d engage blanquita survive re engage blanquita girl ass salcedense believe fact thing warn swear forgive cheat put machete promise course swear do swear swear do stick month be time go father death tenure madness bar exam pass attempt love love shed torture month period fly funeral friend walk beach film want do desperation give beach walk shine sand foot water try hug say do stare rock jut water wind carry hair ride hotel steep pick pair hitchhiker couple love throw car say hotel cry try trick book keep write letter drive work quote compose e  mail disown sucias block e mail change phone number stop drink stop smoke claim re sex addict start attend meeting blame father blame mother blame patriarchy blame find therapist cancel give password e  mail account start take salsa class swear dance claim claim hour clockwork say re try day sit bed say ask move apartment share teach

In [52]:
topic_model.get_top_docs(topic=0, n_docs=2)

[('want visit lady study stroller bag boyfriend baby check phone time go degree cent humidity accord hour way stage have icon event minute break total end walk min train min bus min walk min be effort afternoon abort mission say abort call say re baby  make show phone see number  furnace hold shoulder do humming bounce trick try lull nap eye close breathing slow rivulet drool run spine have go stare phone say have chance night know cook morning click baby stroller pull diaper bag shoulder toss backpack direction come picture set do sit tea watch clock wait arrive pick phone landline person make ring think greet ask autopilot think do thank see be dot ring buzzer click dial tone stand receiver hand know know name ask question air say way make feel be know care take think  stare receiver rest cradle boyfriend father mother sister come find sort entanglement happen lend money cousin tell cousin remove d have look one find page explain terminology middle have word self write block letter o

In [53]:
#try with countvectorizer, no anchors
from corextopic import corextopic as ct

vectorizer = CountVectorizer(max_features=20000,
                             stop_words='english', token_pattern="\\b[a-z][a-z]+\\b",
                             max_df=0.60, binary=True)

doc_word = vectorizer.fit_transform(docs)
words = list(np.asarray(vectorizer.get_feature_names()))

topic_model = ct.Corex(n_hidden=6, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=docs)

topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: shirt,roll,tooth,breath,cheek,wrap,sink,squeeze,skin,grip
1: rise,offer,sight,game,form,wander,ground,remain,presence,enter
2: risk,waste,observation,example,income,persuade,midst,prepare,expense,rider
3: save,rubber,support,society,worth,associate,comfort,task,folk,robe
4: suggest,nakedness,rose,silk,service,horror,stranger,wine,telephone,appeal
5: puke,motif,salesman,goal,poop,amaze,bedclothe,workday,wandering,condense
